In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plot
from scipy.integrate import solve_ivp as ode45 

# Specify simulation parameters
nx = 3 
nu = 2
Tsim = 100
dt = 0.1

# States = [x, y, theta, gamma]
x_init = np.array([10.0, 0.0, 0.0, 0.0])

# Specify desired path
R  = 10
pd = lambda l: np.array([R*np.sin(0.01*l), R*np.cos(0.01*l)])

# TODO: Learn Symbolic operations in Python
pdD = lambda l: np.array([0.01*R*np.cos(0.01*l), -0.01*R*np.sin(0.01*l)])

# Specify robot kinematics
def unicycle(t, x, u):
    return np.array([u[0]*np.cos(x[2]), u[0]*np.sin(x[2]), u[1], u[2]])

# Specify path following controller
def pfController(t,x,funpd,funpdD):
    
    p = x[0:1]
    
    theta = x[2]
    
    gamma = x[3]
    
    vd = 1
    
    R = np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]])
    
    pd = funpd(gamma)

    pdDot = funpdD(gamma)*vd
    
    eps = np.array([0.1, 0])
    
    K = np.eye(2)
    
    invDelta = np.linalg.pinv(np.array([[1.0, eps[1]], [0.0, eps[0]]]))
    
    e = R.T@(p - pd) + eps
    
    u_ff = R.T@pdDot
    
    u = -K@e + u_ff
    
    g_err = 0 # TODO: make the modifications
    
    gamma_dot = vd + g_err
    
    return np.append(u, gamma_dot)
    

In [ ]:
class Log:
    
    def __init__(self):
        self.x = []
        self.u = []
        self.t = []


In [ ]:
# Create a log object
pfLog = Log()

# Simulate the system
timeSpan = np.linspace(0, Tsim, int(Tsim/dt))

x = x_init

for index, t in enumerate(timeSpan):
    u = pfController(t,x,pd,pdD)
    
    pfLog.x.append(x)
    pfLog.t.append(t)
    pfLog.u.append(u)
    
    ode_fun = lambda t,x: unicycle(t,x,u)
    sol = ode45(ode_fun, [index*dt, (index+1)*dt], x)
    
    x = sol.y[:,-1]
    
print("Simulation Complete!")
print(pfLog.x)

# Trajectory tracking simulation
This notebook has a sample simulation for the trajectory tracking example and can serve as a starting point for the development of python simulator and more importantly to setup our simulations for adaptive sampling based approaches. As you can see the implementation is not clean and could lead to many corrections for every simulation that we make with different controller. Hence, the motivation for the decent generic simulator. 

Below I will highlight some problems and questions that I thought was difficult for a user too used to Matlab.

## Questions/Notes:

1. Logging: You can see that I made a simple class to log the results. Now each of the attributes in the class are a **list**. This makes it hard to plot I guess, for example if you see the result of *print(pfLog.x)* you can see that the log is actually a list of **numpy arrays**. This combination of python list and numpy arrays in my opinion, is not nice. Since the indexing and the available methods for lists and numpy arrays are different. 

**Question**: Any suggestions how to adapt it to numpy arrays - out of your experience? I am having problem with appending arrays. May be you have a clue. I am too used to column vector notation of Matlab and in numpy they are just one dimensional arrays. Appending 1D array with another 1D array is throwing me some errors, especially with some dimension mismatches.
